In [5]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Input
from keras.optimizers import Adam

# Load data
new_train_images = np.load('new_train_images.npy')
new_train_labels = np.load('new_train_labels.npy')

# Reshape
new_train_images = new_train_images.reshape(-1, 28, 28, 1)

# generator model
def build_generator(latent_dim):
    generator = Sequential()
    generator.add(Dense(7 * 7 * 64, input_dim=latent_dim))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(Reshape((7, 7, 64)))
    generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    generator.add(LeakyReLU(alpha=0.2))
    generator.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return generator

# discriminator model
def build_discriminator(input_shape):
    discriminator = Sequential()
    discriminator.add(Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    discriminator.add(LeakyReLU(alpha=0.2))
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))
    return discriminator

# GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(latent_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
    return gan

# hyperparameters
latent_dim = 100
image_shape = new_train_images.shape[1:]
generator = build_generator(latent_dim)
discriminator = build_discriminator(image_shape)
gan = build_gan(generator, discriminator)

# Generate random latent
latent_vectors = np.random.normal(size=(len(new_train_images), latent_dim))

# Train
gan.fit(latent_vectors, np.ones(len(new_train_images)), epochs=100, batch_size=32)

# Save
generator.save_weights('generator_weights.h5')


Epoch 1/100
407/407 [==============================] - 9s 9ms/step - loss: 0.5528
Epoch 2/100
407/407 [==============================] - 3s 8ms/step - loss: 0.5146
Epoch 3/100
407/407 [==============================] - 3s 8ms/step - loss: 0.5124
Epoch 4/100
407/407 [==============================] - 3s 8ms/step - loss: 0.5055
Epoch 5/100
407/407 [==============================] - 3s 8ms/step - loss: 0.5015
Epoch 6/100
407/407 [==============================] - 3s 8ms/step - loss: 0.4988
Epoch 7/100
407/407 [==============================] - 3s 8ms/step - loss: 0.4988
Epoch 8/100
407/407 [==============================] - 3s 8ms/step - loss: 0.4957
Epoch 9/100
407/407 [==============================] - 3s 8ms/step - loss: 0.4816
Epoch 10/100
407/407 [==============================] - 3s 9ms/step - loss: 0.4788
Epoch 11/100
407/407 [==============================] - 3s 9ms/step - loss: 0.4778
Epoch 12/100
407/407 [==============================] - 3s 9ms/step - loss: 0.4762
Epoch 13/100
